In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig#, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
#model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
#model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_name = "Qwen/Qwen2.5-7B-Instruct-1M"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config,
                                             torch_dtype="auto",##
                                             device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [7]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
text
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant



In [3]:
import pandas as pd
import os

# 指定文件夹路径
folder_path = 'data'

# 列出文件夹下的所有文件和文件夹
file_names = os.listdir(folder_path)

# 过滤出文件（排除文件夹）
#file_names = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]

# 打印所有文件名
for file_name in file_names:
    data_train_tw = pd.read_csv(os.path.join(folder_path, file_name), encoding = 'utf-8-sig')
    print(os.path.join(folder_path, file_name))

data/Public_PriceAndCost_Negative.csv


In [18]:
#for llama
def generate_prompt(content):
    begin = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    syst = "You are a helpful AI assistant that answers questions briefly and directly."
    inst = "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nThe following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, in the formate like:\nkeyword one, keyword two. \nDo not answer anything else.\nText: "+content
    end = "<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n"
    prompt = ("").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['text_cleaned'][1]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
The following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, in the formate like:
keyword one, keyword two. 
Do not answer anything else.
Text: Earlier this week, TSB Chair Kathy Fox presented at the  flight instructor refresher course, touching on frequent general aviation accident scenarios and the TSB’s Air Taxi Safety Issue Investigation <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>




In [11]:
#for Qwen
def generate_prompt(content):
    begin = "<|im_start|>system\n"
    syst = "You are a helpful AI assistant that answers questions briefly and directly."
    inst = "<|im_end|>\n<|im_start|>user\nThe following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, do not answer anything else.\nText: "+content
    end = "<|im_end|>\n<|im_start|>assistant\n"
    prompt = ("").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['text_cleaned'][1]))

<|im_start|>system
You are a helpful AI assistant that answers questions briefly and directly.<|im_end|>
<|im_start|>user
The following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, do not answer anything else.
Text:  they found it now in atlanta altho still not delivered. Unfortunately it had to be delivered by 4pm Monday 12/21 so it is void on arrival. Fortunately i received new material via fed express and returned via delta cargo and air taxi on monday or else i was SOL. Cost me over $100<|im_end|>
<|im_start|>assistant



In [10]:
#for deepseek Qwen
def generate_prompt(content):
    begin = "<｜begin▁of▁sentence｜>"
    syst = "You are a helpful AI assistant that answers questions briefly and directly."
    inst = "\n<｜User｜>The following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, do not answer anything else.\nText: "+content
    end = "\n<｜Assistant｜><think>\nOkay, so I need to figure out the keywords for the reasons of dissatisfaction in the given text.\nAnd I shoud only answer the keywords, in the formate like:\nkeyword one, keyword two.\n</think>\n\nThe dissatisfaction keywords are:"
    #Okay, so I need to figure out how to approach this task. The user has provided a query where they want me to analyze given texts related to urban low-altitude transportation. They want each text ranked for attitude (positive, negative, neutral) and categorized into specific topics.\n\nFirst, I should understand the structure. Each response is just two things: the attitude and the category number, without any extra explanation.\n\nI also need to make sure not to add any extra information, just the required parts. So, keeping responses concise is key.\n\nAlright, I think I'm ready to apply this method to other texts as needed.\n</think>"
    prompt = ("").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['text_cleaned'][1]))

<｜begin▁of▁sentence｜>You are a helpful AI assistant that answers questions briefly and directly.
<｜User｜>The following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, do not answer anything else.
Text:  they found it now in atlanta altho still not delivered. Unfortunately it had to be delivered by 4pm Monday 12/21 so it is void on arrival. Fortunately i received new material via fed express and returned via delta cargo and air taxi on monday or else i was SOL. Cost me over $100
<｜Assistant｜><think>
Okay, so I need to figure out the keywords for the reasons of dissatisfaction in the given text.
And I shoud only answer the keywords, in the formate like:
keyword one, keyword two.
</think>

The dissatisfaction keywords are:


In [12]:
text = generate_prompt(data_train_tw['text_cleaned'][1])
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generation_output = model.generate(
    **model_inputs,
    max_new_tokens=16,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

/venv/main/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


system
You are a helpful AI assistant that answers questions briefly and directly.
user
The following texts are research or tweets about urban low-altitude transportation. Give the keywords for the reasons for dissatisfaction. Only answer the keywords, do not answer anything else.
Text:  they found it now in atlanta altho still not delivered. Unfortunately it had to be delivered by 4pm Monday 12/21 so it is void on arrival. Fortunately i received new material via fed express and returned via delta cargo and air taxi on monday or else i was SOL. Cost me over $100
assistant
delivery, arrival, cost


In [ ]:
import os
import pandas as pd
import numpy as np
import time

# 指定文件夹路径
folder_path = 'data'

# 列出文件夹下的所有文件和文件夹
file_names = os.listdir(folder_path)

# 过滤出文件（排除文件夹）
#file_names = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]

# 打印所有文件名
for file_name in file_names:
    data_train_tw = pd.read_csv(os.path.join(folder_path, file_name), encoding = 'utf-8-sig')
    
    data = []
    save_interval = 100
    
    start_time = time.time()  # 记录开始时间
    
    for i in range(len(data_train_tw)):#['text_cleaned']):
    #if i % 1 == 0:
        #if pd.isna(data_train_tw.iloc[i]['result']) and not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
        if not pd.isna(data_train_tw.iloc[i]['text_cleaned']):
            #print(i)
            
            input_prompt = generate_prompt(data_train_tw.iloc[i]['text_cleaned'])
            model_inputs = tokenizer([input_prompt], return_tensors="pt").to(model.device)

            generation_output = model.generate(
                **model_inputs,
                max_new_tokens=8,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            
            op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            #print(op)

            #inst_index = op.find('</think>\n\nThe dissatisfaction keywords are: ')
            inst_index = op.find('assistant\n')
        
            if inst_index != -1:
                #print(text)
                #print(op[inst_index + len('assistant\n'):])
                data.append({"number": i, "link": data_train_tw.iloc[i]['link'], "text": data_train_tw.iloc[i]['text_cleaned'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
                #data.append({"number": i, "link": data_train_tw.iloc[i]['Title'], "text": data_train_tw.iloc[i]['Abstract'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
                #print(data[-1])
            else:
                print("未找到'assistant'标记")
                #data.append({"number": i, "text": text, "sentiment":""})
    
            # 每 save_interval 个迭代保存一次
            if len(data) == save_interval:
                df = pd.DataFrame(data)
                df.to_csv(file_name[:3]+"_output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)  # 追加模式
                data = []
                end_time = time.time()  # 记录结束时间
                elapsed_time = end_time - start_time  # 计算用时
                print(f"已保存到 output.csv，当前迭代次数：{i}，用时：{elapsed_time:.2f} 秒")
        
                start_time = end_time  # 更新开始时间，用于计算下一个周期的用时

    # 最后一次保存
    df = pd.DataFrame(data)
    df.to_csv(file_name[:3]+"_output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)
    print("已保存到 output_tw.csv")



/venv/main/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [4]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [5]:
text

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n'

In [18]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Okay, so I need to figure out the keywords for the reasons of dissatisfaction in the given text. Let me read through the text carefully.

The text says: "they found it now in Atlanta altho still not delivered. Unfortunately it had to be delivered by 4pm Monday 12/21 so it is void on arrival. Fortunately i received new material via fed express and returned via delta cargo and air taxi on monday or else i was SOL. Cost me over $100."

Alright, the user is talking about a delivery issue. They were supposed to get something delivered by 4pm on Monday, 12/21, but it wasn't delivered. Because of that, it's void upon arrival, which probably means it's no longer valid or useful. They had to get new materials through FedEx and then return the old ones via Delta Cargo and Air Taxi on Monday. If they hadn't done that, they were SOL, which I think stands for "shit out of luck," meaning they would have been in a bad situation. The cost was over $100.

So, the dissatisfaction here is clearly about t

In [3]:
import pandas as pd

data_train_tw = pd.read_csv('unidentify_sentiment_data(1).csv', encoding = 'utf-8-sig')
print(data_train_tw)

       Unnamed: 0      keyword  \
0            1642     air taxi   
1            2334     air taxi   
2           25461     air taxi   
3           28169     air taxi   
4           42239  drone taxis   
...           ...          ...   
10238      447506         vtol   
10239      447519         vtol   
10240      447520         vtol   
10241      447521         vtol   
10242      447538         vtol   

                                                    link       user_id  \
0      https://twitter.com/brice402/status/1296728841...      7.91e+17   
1      https://twitter.com/America_1always/status/126...   230328648.0   
2      https://twitter.com/ruth_joyner/status/1695947...  2608369436.0   
3      https://twitter.com/jonnyknocksvil1/status/161...       1.3e+18   
4      https://twitter.com/bobdurantjones/status/1582...      1.54e+18   
...                                                  ...           ...   
10238  https://twitter.com/LeMoine_Russell/status/174...      1.71E+18   

In [4]:
def generate_prompt(content):
    begin = "<｜begin▁of▁sentence｜>system\n"
    #syst = "<<SYS>> You are a helpful AI assistant that answers questions briefly and directly.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "You are a helpful AI assistant that answers questions briefly and directly.\n"
    inst = "user\nThe following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: \n1: Safety \n2: Aerodynamics \n3: Integration and Infrastructure \n4: Automation \n5: Price and cost \n6: Policy \n7: User experience \nOnly answer the attitude and category number, don't answer anything else. For example: Positive, 5\nText: "+content
    end = "\nassistant\n<think>\nOkay, so I need to figure out how to approach this task. The user has provided a query where they want me to analyze given texts related to urban low-altitude transportation. They want each text ranked for attitude (positive, negative, neutral) and categorized into specific topics.\n\nFirst, I should understand the structure. Each response is just two things: the attitude and the category number, without any extra explanation.\n\nI also need to make sure not to add any extra information, just the required parts. So, keeping responses concise is key.\n\nAlright, I think I'm ready to apply this method to other texts as needed.\n</think>"
    prompt = (" ").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['text_cleaned'][1]))

<｜begin▁of▁sentence｜>system
 You are a helpful AI assistant that answers questions briefly and directly.
 user
The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: 
1: Safety 
2: Aerodynamics 
3: Integration and Infrastructure 
4: Automation 
5: Price and cost 
6: Policy 
7: User experience 
Only answer the attitude and category number, don't answer anything else. For example: Positive, 5
Text:      I guess at some point you have to ask why a sex addict like Clinton, needed 26 trips on Epstein’s private plane?  Maybe Epstein ran a air taxi service!  Haha 
assistant
<think>
Okay, so I need to figure out how to approach this task. The user has provided a query where they want me to analyze given texts related to urban low-altitude transportation. They want each text ranked for attitude (positive, negative, neutral) and categorized

In [12]:
input_prompt = generate_prompt(data_train_tw['text_cleaned'][1])
#input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
#inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
input_tokens = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
with torch.cuda.amp.autocast():
    generation_output = model.generate(
        input_ids=input_tokens,
        attention_mask=attention_mask,
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

system
 You are a helpful AI assistant that answers questions briefly and directly.
 user
The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: 
1: Safety 
2: Aerodynamics 
3: Integration and Infrastructure 
4: Automation 
5: Price and cost 
6: Policy 
7: User experience 
Only answer the attitude and category number, don't answer anything else. For example: Positive, 5
Text:      I guess at some point you have to ask why a sex addict like Clinton, needed 26 trips on Epstein’s private plane?  Maybe Epstein ran a air taxi service!  Haha 
assistant
<think>
Okay, so I need to figure out how to approach this task. The user has provided a query where they want me to analyze given texts related to urban low-altitude transportation. They want each text ranked for attitude (positive, negative, neutral) and categorized into specific topics

In [5]:
input_prompt = generate_prompt(data_train_tw['text_cleaned'][3])
#input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
#inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
input_tokens = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
with torch.cuda.amp.autocast():
    generation_output = model.generate(
        input_ids=input_tokens,
        attention_mask=attention_mask,
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


system
 You are a helpful AI assistant that answers questions briefly and directly.
 user
The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: 
1: Safety 
2: Aerodynamics 
3: Integration and Infrastructure 
4: Automation 
5: Price and cost 
6: Policy 
7: User experience 
Only answer the attitude and category number, don't answer anything else. For example: Positive, 5
Text:  I say put hunter on an air taxi with a couple of hookers and see what happens. 
assistant
<think>
Okay, so I need to figure out how to approach this task. The user has provided a query where they want me to analyze given texts related to urban low-altitude transportation. They want each text ranked for attitude (positive, negative, neutral) and categorized into specific topics.

First, I should understand the structure. Each response is just two things: the 

In [7]:
print(tokenizer.special_tokens_map)

{'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}


In [ ]:
import pandas as pd
import numpy as np
import time

data = []
save_interval = 100

start_time = time.time()  # 记录开始时间

for i in range(len(data_train_tw)):#['text_cleaned']):
    if i % 1 == 0:
        #if pd.isna(data_train_tw.iloc[i]['result']) and not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
        if not pd.isna(data_train_tw.iloc[i]['text_cleaned']):
            #print(i)
            
            input_prompt = generate_prompt(data_train_tw.iloc[i]['text_cleaned'])
            #input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
            #inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
            inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
            input_tokens = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]
            with torch.cuda.amp.autocast():
                generation_output = model.generate(
                    input_ids=input_tokens,
                    attention_mask=attention_mask,
                    max_new_tokens=8,
                    do_sample=False,
                    repetition_penalty=1.1,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id
                )
            op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            #print(op)

            inst_index = op.find('</think>\n\n')
        
            if inst_index != -1:
                #print(text)
                #print(op[inst_index + len('assistant\n'):])
                data.append({"number": i, "link": data_train_tw.iloc[i]['link'], "text": data_train_tw.iloc[i]['text_cleaned'], "result": op[inst_index + len('</think>\n\n'):].replace('\n', ' ')})
                #data.append({"number": i, "link": data_train_tw.iloc[i]['Title'], "text": data_train_tw.iloc[i]['Abstract'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
                #print(data[-1])
            else:
                print("未找到'assistant\n'标记")
                #data.append({"number": i, "text": text, "sentiment":""})
    
            # 每 save_interval 个迭代保存一次
            if len(data) == save_interval:
                df = pd.DataFrame(data)
                df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)  # 追加模式
                data = []
                end_time = time.time()  # 记录结束时间
                elapsed_time = end_time - start_time  # 计算用时
                print(f"已保存到 output.csv，当前迭代次数：{i}，用时：{elapsed_time:.2f} 秒")
        
                start_time = end_time  # 更新开始时间，用于计算下一个周期的用时

# 最后一次保存
df = pd.DataFrame(data)
df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)
print("已保存到 output_tw.csv")

已保存到 output.csv，当前迭代次数：99，用时：89.10 秒
已保存到 output.csv，当前迭代次数：199，用时：90.41 秒
已保存到 output.csv，当前迭代次数：299，用时：89.51 秒
已保存到 output.csv，当前迭代次数：399，用时：89.07 秒
已保存到 output.csv，当前迭代次数：499，用时：89.15 秒
已保存到 output.csv，当前迭代次数：599，用时：89.68 秒
已保存到 output.csv，当前迭代次数：699，用时：89.21 秒


In [21]:
data

[{'number': 0,
  'link': 'https://twitter.com/brice402/status/1296728841789026304',
  'text': "   If the fat ass in the white house can vote by mail then everyone can. Him being in the white house is no excuse because he's down at Mar-A-Lago golfing just about every week and he uses the taxpayers Air Force 1 for his phucking pleasure trips like it's his own air taxi.",
  'result': 'Negative, 6'},
 {'number': 1,
  'link': 'https://twitter.com/America_1always/status/1263267155338035200',
  'text': '     I guess at some point you have to ask why a sex addict like Clinton, needed 26 trips on Epstein’s private plane?  Maybe Epstein ran a air taxi service!  Haha',
  'result': 'Negative, 6'},
 {'number': 2,
  'link': 'https://twitter.com/ruth_joyner/status/1695947931533402283',
  'text': "            The Trump's actually do legal business. They are smart. The Biden's? No legal business. Bribery, sleazy treasonous deals making millions. 20 shell company's, Air Force 2 Hunters Air taxi at our e

In [2]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [3]:
import pandas as pd

data_train_tw = pd.read_csv('academic_paper_2015-2024.csv', encoding = 'utf-8-sig')
print(data_train_tw)

                                                Authors  \
0             Samadzad M.; Ansari F.; Afshari Moez M.A.   
1                      Kim S.-W.; Kwon D.-H.; Cho I.-H.   
2                    Qin V.L.; Ding G.; Balakrishnan H.   
3                                    Sah B.; Titiyal R.   
4     Di Mascio P.; Celesti M.; Sabatini M.; Moretti L.   
...                                                 ...   
2543                                            Do K.D.   
2544      Sinha P.; Stoll A.M.; Stilson E.V.; Bevirt J.   
2545               Güçlü A.; Arıkan K.B.; Kurtuluş D.F.   
2546        Oberschwendtner S.; Roessler C.; Hornung M.   
2547  Yang Y.; Karimadini M.; Xiang C.; Teo S.H.; Ch...   

                                      Author full names  \
0     Samadzad, Mahdi (50861885300); Ansari, Fatemeh...   
1     Kim, Seon-Woong (59347621400); Kwon, Do-Hun (5...   
2     Qin, Victor L. (57446368000); Ding, Geoffrey (...   
3     Sah, Bhawesh (57195055488); Titiyal, Rohit (57...

In [4]:
data_train_tw['cleaned_text'] = data_train_tw['Title'] + data_train_tw['Abstract']

In [6]:
def generate_prompt(content):
    begin = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
    #syst = "<<SYS>> You are a helpful AI assistant that answers questions briefly and directly.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>"
    inst = "The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: \n1: Safety \n2: Aerodynamics \n3: Integration and Infrastructure \n4: Automation \n5: Price and cost \n6: Policy \n7: User experience \nOnly answer the attitude and category number, don't answer anything else. For example: Positive, 5\nText: "+content
    end = "<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"
    prompt = (" ").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['cleaned_text'][1]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>
<|start_header_id|>user<|end_header_id|> The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: 
1: Safety 
2: Aerodynamics 
3: Integration and Infrastructure 
4: Automation 
5: Price and cost 
6: Policy 
7: User experience 
Only answer the attitude and category number, don't answer anything else. For example: Positive, 5
Text: Temperature Management Strategy for Urban Air Mobility Batteries to Improve Energy Efficiency in Low-Temperature ConditionsAs urban population concentration accelerates, issues such as traffic congestion caused by automobiles and climate change due to carbon dioxide emissions are becoming increasingly severe. Recently, urban air mobility (UAM) has been attracting 

In [ ]:
import pandas as pd
import numpy as np
import time

data = []
save_interval = 20

start_time = time.time()  # 记录开始时间

for i in range(len(data_train_tw)):#['text_cleaned']):
    if i % 1 == 0:
        #if pd.isna(data_train_tw.iloc[i]['result']) and not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
        if not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
            #print(i)
            
            input_prompt = generate_prompt(data_train_tw.iloc[i]['cleaned_text'])
            #input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
            #inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
            inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
            input_tokens = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]
            with torch.cuda.amp.autocast():
                generation_output = model.generate(
                    input_ids=input_tokens,
                    attention_mask=attention_mask,
                    max_new_tokens=8,
                    do_sample=False,
                    repetition_penalty=1.1,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id
                )
            op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            #print(op)

            inst_index = op.find('assistant\n')
        
            if inst_index != -1:
                #print(text)
                #print(op[inst_index + len('assistant\n'):])
                #data.append({"number": i, "link": data_train_tw.iloc[i]['link'], "text": data_train_tw.iloc[i]['cleaned_text'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
                data.append({"number": i, "link": data_train_tw.iloc[i]['Title'], "text": data_train_tw.iloc[i]['Abstract'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
            else:
                print("未找到'assistant\n'标记")
                #data.append({"number": i, "text": text, "sentiment":""})
    
            # 每 save_interval 个迭代保存一次
            if len(data) == save_interval:
                df = pd.DataFrame(data)
                df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)  # 追加模式
                data = []
                end_time = time.time()  # 记录结束时间
                elapsed_time = end_time - start_time  # 计算用时
                print(f"已保存到 output.csv，当前迭代次数：{i}，用时：{elapsed_time:.2f} 秒")
        
                start_time = end_time  # 更新开始时间，用于计算下一个周期的用时

# 最后一次保存
df = pd.DataFrame(data)
df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)
print("已保存到 output_tw.csv")
            

已保存到 output.csv，当前迭代次数：19，用时：9.96 秒
已保存到 output.csv，当前迭代次数：39，用时：9.99 秒
已保存到 output.csv，当前迭代次数：59，用时：9.87 秒
已保存到 output.csv，当前迭代次数：79，用时：9.92 秒
已保存到 output.csv，当前迭代次数：99，用时：9.94 秒
已保存到 output.csv，当前迭代次数：119，用时：9.66 秒
已保存到 output.csv，当前迭代次数：139，用时：9.98 秒
已保存到 output.csv，当前迭代次数：159，用时：9.94 秒
已保存到 output.csv，当前迭代次数：179，用时：9.83 秒
已保存到 output.csv，当前迭代次数：199，用时：9.96 秒
已保存到 output.csv，当前迭代次数：219，用时：9.88 秒
已保存到 output.csv，当前迭代次数：239，用时：9.86 秒
已保存到 output.csv，当前迭代次数：259，用时：9.80 秒
已保存到 output.csv，当前迭代次数：279，用时：9.93 秒
已保存到 output.csv，当前迭代次数：299，用时：10.23 秒
已保存到 output.csv，当前迭代次数：319，用时：9.97 秒
已保存到 output.csv，当前迭代次数：339，用时：9.96 秒
已保存到 output.csv，当前迭代次数：359，用时：9.95 秒
已保存到 output.csv，当前迭代次数：379，用时：9.87 秒
已保存到 output.csv，当前迭代次数：399，用时：9.85 秒
已保存到 output.csv，当前迭代次数：419，用时：10.08 秒
已保存到 output.csv，当前迭代次数：439，用时：9.92 秒


In [4]:
data = load_dataset("enyuan/Abstracts")
data_train = data["train"]

custom_data = load_dataset('json', data_files='data_eval.json')
data_val = custom_data['train']

# Print the dataset details
print(data_train)
print(data_val)

# Access an example
#example = data_train[0]
#print(example)

def generate_prompt(title, abstract=None, eos_token="</s>"):
  instruction = "The abstract of the paper:\n"
  input = f"{title}\n"
  abstract = f"Abstract: {abstract + ' ' + eos_token if abstract else ''} "
  prompt = (" ").join([instruction, input, abstract])
  return prompt

print(generate_prompt(data_train[0]["title"], data_train[0]["abstract"]))

Dataset({
    features: ['title', 'doi', 'abstract', 'publicationDate'],
    num_rows: 165071
})
Dataset({
    features: ['title', 'abstract', 'publicationDate'],
    num_rows: 559
})
The abstract of the paper:
 Inconel 625 sustainable milling surface integrity and the dependence on alloy processing route
 Abstract: The discovery of deepwater oil and gas sources has altered the scenario of world production of oil products, attracting even more attention to nickel superalloys. However, this class of materials can be used in several applications. Furthermore, nickel superalloys are highly dependent on their processing history, and the manner in which superalloys react to machining can directly affect the finished product. This work aims to evaluate the surface integrity of two different materials after cryogenic side-milling in conditions that stimulate severe plastic deformation (SPD) and high heat generation. The results show that the material response to machining depends strongly on 

In [4]:
input_prompt = generate_prompt(data_train[50]["title"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
  generation_output = model.generate(
      input_ids=input_tokens,
      max_new_tokens=1000,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=0.3,
      repetition_penalty=1.15,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

The abstract of the paper:
 Effect of cryogenic cooling on residual stresses and surface finish of 316L during hybrid manufacturing
 Abstract:   In this work, a novel approach for reducing the residual stress in the welded joints of stainless steel is presented. A new process called Hybrid Manufacturing (HM) was developed to reduce the residual stress in the welded joints by using two different techniques namely, laser beam welding (LBW) and cryogenic treatment (CT). The effectiveness of HM technique has been studied with respect to the reduction of residual stress and improvement in surface roughness. The results showed that the residual stress can be reduced up to 40% when compared to conventional LBW method. Moreover, the surface roughness can also be improved significantly as shown by the Ra value which decreases from 25.87µm to 19.31µm after CT.
The full text of the article: http://www.sciencedirect.com/science/article/pii/S092583881400132X


In [4]:
with open('materials.txt', 'r') as file:
    word_list = file.read().splitlines()